# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [2]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df['col2'] = [5,6,7,8]
df

,col1,col2
0,1,5
1,2,6
2,3,7
3,4,8


## 2. Deleting a row in a DataFrame

In [3]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df.drop('d', inplace=True)
df

,col1
a,1
b,2
c,3


## 3. Creating a DataFrame from a few Series

In [4]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))
pd.DataFrame({'ser_1': ser_1, 'ser_2': ser_2, 'ser_3': ser_3})

,ser_1,ser_2,ser_3
0,-0.673460,-0.375016,0.667032
1,0.902934,-0.757647,1.186403
2,0.225045,0.002153,-1.096205
3,0.180715,-0.563616,0.358734
4,-0.898724,0.268541,1.129626
5,0.123984,-1.014790,-0.653534


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [5]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df

,col_1,col_2,col_3
obs1,0.12,0.9,NaN
obs2,7.00,9.0,NaN
obs3,45.00,34.0,NaN
obs4,10.00,11.0,NaN


In [6]:
df.col_2

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [7]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.loc['obs3']

col_1    45.0
col_2    34.0
col_3     NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [8]:
# using the same DataFrame, index into into its first row
df.iloc[:1]

,col_1,col_2,col_3
obs1,0.12,0.9,NaN


# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [9]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'], engine='python')

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'], engine='python')

## 2. How to load the training and testing subsets

In [10]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0)
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0)

In [11]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [12]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [13]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [14]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [15]:
# This is a baseline that just gives an average rating to everything
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [16]:
print('RMSE for my estimate function: %s' % evaluate(my_estimate_func))

RMSE for my estimate function: 1.2323719526527521


The above is the starting baseline. 
Want to improve from here.  
0 error means the recommendation is perfect.
4 (5-1) is the maximum amount it could be off.

# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [17]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    """ Simple collaborative filter based on mean ratings. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty:
        return 3.0
    else:
        return ratings_by_others.rating.mean()

    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

4.0

In [18]:
print(f'RMSE for estimate2 is: {evaluate(collab_mean)}.')

RMSE for estimate2 is: 1.1234279896011794.


# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [19]:
user_info = users.set_index('user_id')
user_info.head(3)

,gender,age,occupation,zip
user_id,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117


In [20]:
class CollaborativeRecommendation:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def __init__(self):
        """ Prepare data structures for estimation. """
        self.means_by_gender = movielens_train.pivot_table('rating', index='movie_id', columns='gender')
        self.means_by_age = movielens_train.pivot_table('rating', index='movie_id', columns='age')
        self.means_by_zip = movielens_train.pivot_table('rating', index='movie_id', columns='zip')
        self.means_by_occupation = movielens_train.pivot_table('rating', index='movie_id', columns='occupation')

    def estimate_gender(self, user_id, movie_id):
        """ Mean ratings by other users of the same gender. """
        
        if movie_id not in self.means_by_gender.index: 
            return 3.0
        
        user_gender = user_info.loc[user_id, 'gender']
        if ~np.isnan(self.means_by_gender.loc[movie_id, user_gender]):
            return self.means_by_gender.loc[movie_id, user_gender]
        else:
            return self.means_by_gender.loc[movie_id].mean()

    def estimate_age(self, user_id, movie_id):
        """ Mean ratings by other users of the same age. """
        
        if movie_id not in self.means_by_age.index: 
            return 3.0
        
        user_age = user_info.loc[user_id, 'age']
        if ~np.isnan(self.means_by_age.loc[movie_id, user_age]):
            return self.means_by_age.loc[movie_id, user_age]
        else:
            return self.means_by_age.loc[movie_id].mean()

    def estimate_zip(self, user_id, movie_id):
        """ Mean ratings by other users of the same zip code. """
        
        if movie_id not in self.means_by_zip.index: 
            return 3.0
        
        user_zip = user_info.loc[user_id, 'zip']
        if ~np.isnan(self.means_by_zip.loc[movie_id, user_zip]):
            return self.means_by_zip.loc[movie_id, user_zip]
        else:
            return self.means_by_zip.loc[movie_id].mean()

    def estimate_occupation(self, user_id, movie_id):
        """ Mean ratings by other users of the same occupation. """
        
        if movie_id not in self.means_by_occupation.index: 
            return 3.0
        
        user_occupation = user_info.loc[user_id, 'occupation']
        if ~np.isnan(self.means_by_occupation.loc[movie_id, user_occupation]):
            return self.means_by_occupation.loc[movie_id, user_occupation]
        else:
            return self.means_by_occupation.loc[movie_id].mean()

In [23]:
reco = CollaborativeRecommendation()
print('RMSE for Gender: %s' % evaluate(reco.estimate_gender))
print('RMSE for Age: %s' % evaluate(reco.estimate_age))
print('RMSE for Zip: %s' % evaluate(reco.estimate_zip))
print('RMSE for Occupation: %s' % evaluate(reco.estimate_occupation))

RMSE for Gender: 1.1740082417112805
RMSE for Age: 1.2052013344107597
RMSE for Zip: 1.1256640319176328
RMSE for Occupation: 1.202876964364191


In [32]:
class CollaborativeRecommendation2:
    """ Collaborative filtering using an implicit sim(u,u'). """

    def __init__(self, feature):
        """ Prepare data structures for estimation. """
        self._feature = feature
        self.means_by_feature = movielens_train.pivot_table('rating', index='movie_id', columns=self.feature)

    @property
    def feature(self):
        return self._feature
        
    def estimate(self, user_id, movie_id):
        """ Mean ratings by other users of the same feature. """
        
        if movie_id not in self.means_by_feature.index: 
            return 3.0
        
        user_feature = user_info.loc[user_id, self.feature]
        if ~np.isnan(self.means_by_feature.loc[movie_id, user_feature]):
            return self.means_by_feature.loc[movie_id, user_feature]
        else:
            return self.means_by_feature.loc[movie_id].mean()

In [33]:
gender = CollaborativeRecommendation2('gender')
print('RMSE for Gender: %s' % evaluate(gender.estimate))
age = CollaborativeRecommendation2('age')
print('RMSE for Age: %s' % evaluate(age.estimate))
zip_code = CollaborativeRecommendation2('zip')
print('RMSE for Zip: %s' % evaluate(zip_code.estimate))
occupation = CollaborativeRecommendation2('occupation')
print('RMSE for Occupation: %s' % evaluate(occupation.estimate))

RMSE for Gender: 1.1740082417112805
RMSE for Age: 1.2052013344107597
RMSE for Zip: 1.1256640319176328
RMSE for Occupation: 1.202876964364191


# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [24]:
from scipy.special import logsumexp

# Euclidean 'similarity'
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

# Cosine similarity
def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

# Pearson correlation
def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(logsumexp(s1_c ** 2)) * np.sum(logsumexp(s2_c ** 2)))

# Jaccard similarity
def jaccard(s1, s2):
    dotp = np.sum(s1 * s2)
    return dotp / (np.sum(s1 ** 2) + np.sum(s2 ** 2) - dotp)

def binjaccard(s1, s2):
    dotp = s1.index.intersection(s2.index).size
    return dotp / (s1.sum() + s2.sum() - dotp)

In [25]:
class CollaborativeSimilarityRecommendation:
    """ Collaborative filtering using a custom sim(u,u'). """

    def __init__(self, similarity=None):
        """ Prepare datastructures for estimation. """
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')
        self._similarity = similarity
        
    @property
    def similarity(self):
        return self._similarity
    
    @similarity.setter
    def similarity(self, value):
        self._similarity = value
        
    def estimate(self, user_id, movie_id):
        """ Ratings weighted by correlation similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: self.similarity(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
        

In [26]:
reco = CollaborativeSimilarityRecommendation(pearson)
print('RMSE for Pearson: %s' % evaluate(reco.estimate))

RMSE for Pearson: 1.1234279896011794


In [27]:
reco = CollaborativeSimilarityRecommendation(euclidean)
print('RMSE for Euclidean: %s' % evaluate(reco.estimate))

RMSE for Euclidean: 1.1234294760026475


In [28]:
reco = CollaborativeSimilarityRecommendation(cosine)
print('RMSE for Cosine: %s' % evaluate(reco.estimate))

RMSE for Cosine: 1.1343992169551975


In [29]:
reco = CollaborativeSimilarityRecommendation(jaccard)
print('RMSE for Jaccard: %s' % evaluate(reco.estimate))

RMSE for Jaccard: 1.1343992169551975


In [30]:
reco = CollaborativeSimilarityRecommendation(binjaccard)
print('RMSE for Bin Jaccard: %s' % evaluate(reco.estimate))

RMSE for Bin Jaccard: 1.1234279896011794


In [31]:
from functools import reduce

class Recommender(object):
    def __init__(self):
        self.overall_mean = movielens_train['rating'].mean()
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate_movie(self, user_id, movie_id):
        all_ratings = movielens_train.loc[movielens_train.movie_id == movie_id]
        if all_ratings.empty:
            return self.overall_mean
        all_ratings.set_index('user_id', inplace=True)
        their_ids = all_ratings.index
        their_ratings = all_ratings.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: pearson(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)

    def estimate_user(self, user_id):
        similar = movielens_train.loc[reduce(np.logical_and,
                                             [movielens_train.gender == users.loc[user_id].gender,
                                              movielens_train.occupation == users.loc[user_id].occupation,
                                              abs(movielens_train.age - users.loc[user_id].age) < 10 ])]

        if similar.empty:
            return self.overall_mean
        else:
            return similar['rating'].mean()

    def estimate(self, user_id, movie_id):
        return 0.5 * self.estimate_user(user_id) + 0.5 * self.estimate_movie(user_id, movie_id)

rec = Recommender()
print('RMSE for recommender estimate class: %s' % evaluate(rec.estimate))


RMSE for recommender estimate class: 1.0522288050053856
